#### Pricing of SS Jewel Exchange Ticket

![alt-text](assets/daily_dungeon.PNG)

Assume player buys blue, green, orange jewel per daily dungeon run, how much mesos is required to get a SS jewel
* Blue: 3.5M
* Green: 0.5M
* Orange: 3.5M

In [1]:
from random import random
import numpy as np

rng = lambda thresh: random() <= thresh

In [2]:
ORNAGE_COST = 3.5
BLUE_COST = 3.5
GREEN_COST = 0.5

FUSE_S_COST = 0.09
FUSE_SS_COST = 0.27

ORANGE_THRESH = 0.1
BLUE_THRESH = 0.3

FUSE_S_PROB = 0.1
FUSE_SS_PROB = 0.05

class Jewel:
    def __init__(self, use_orange=True, use_green=True, use_blue=True, verbose=False):
        self.mesos_spent = 0
        self.dungeon_run = 0
        
        self.orange_jewel = 0
        self.green_jewel = 0
        self.blue_jewel = 0
        
        self.use_orange = use_orange
        self.use_green = use_green
        self.use_blue = use_blue
        
        self.use_lucky_for_s = True
        self.use_lucky_for_ss = True
        
        self.a_class = 0
        self.s_class = 0
        self.ss_class = 0
        
        self.lucky_3pct = 0
        self.lucky_5pct = 0
        
        self.fuse_s_attempt = 0
        self.fuse_ss_attempt = 0
        
        self.verbose = verbose
        
    def ss_fused(self):
        return self.ss_class != 0
        
    def run_daily_dungeon(self):
        self.a_class += 1
        self.dungeon_run += 1
        
        if self.use_orange:
            self.mesos_spent += ORNAGE_COST
            self.s_class += rng(ORANGE_THRESH)
            
        if self.use_green:
            self.mesos_spent += GREEN_COST
            self.a_class += 1
            
        if self.use_blue:
            self.mesos_spent += BLUE_COST
            even_prob = 1 / 4
            if rng(BLUE_THRESH):
                rn = random()
                if rn <= 0.25:
                    self.a_class += 1
                elif rn <= 0.5:
                    self.s_class += 1
                elif rn <= 0.75:
                    self.lucky_3pct += 1
                else:
                    self.lucky_5pct += 1
    
    def can_fuse_s(self):
        return self.a_class >= 3
    
    def can_fuse_ss(self):
        return self.s_class >= 3
    
    def use_lucky(self, chance):
        # prioritize 3% lucky jewel
        if self.lucky_3pct > 0:
            self.lucky_3pct -= 1
            chance += 0.03
        elif self.lucky_5pct > 0:
            self.lucky_5pct -= 1
            chance += 0.05
        return chance
        
    def fuse_s(self):
        self.a_class -= 3
        chance = FUSE_S_PROB
        if self.use_lucky_for_s:
            chance = self.use_lucky(chance)
        self.s_class += rng(chance)
        self.mesos_spent += FUSE_S_COST
        self.fuse_s_attempt += 1
        
        if self.verbose:
            print(f"fuse_s: {self.s_class}, chance={chance}")
    
    def fuse_ss(self):
        self.s_class -= 3
        chance = FUSE_SS_PROB
        if self.use_lucky_for_ss:
            chance = self.use_lucky(chance)
        self.ss_class += rng(chance)
        self.mesos_spent += FUSE_SS_COST
        self.fuse_ss_attempt += 1
        
        if self.verbose:
            print(f"fuse_ss: {self.ss_class}, chance={chance}")
        
    def fuse_jewel(self):
        while self.can_fuse_s():
            self.fuse_s()
        while self.can_fuse_ss():
            self.fuse_ss()
    
    def __str__(self):
        return f"""DailyDungeon(
        a_class={self.a_class}, 
        s_class={self.s_class}, 
        ss_class={self.ss_class}, 
        run={self.dungeon_run}, 
        mesos={self.mesos_spent}, 
        lucky_5pct={self.lucky_5pct}, 
        lucky_3pct={self.lucky_3pct},
        s_attempt={self.fuse_s_attempt},
        ss_attempt={self.fuse_ss_attempt})"""

In [3]:
a = Jewel()

In [4]:
a.run_daily_dungeon()
a.fuse_jewel()
print(a)

DailyDungeon(
        a_class=2, 
        s_class=1, 
        ss_class=0, 
        run=1, 
        mesos=7.5, 
        lucky_5pct=0, 
        lucky_3pct=0,
        s_attempt=0,
        ss_attempt=0)


#### Monte Carlo Simulation
    

In [5]:
trials = 10000
s_attempt_arr = np.zeros(trials)
ss_attempt_arr = np.zeros(trials)
run_arr = np.zeros(trials)
mesos_arr = np.zeros(trials)

for i in range(trials):
    state = Jewel(verbose=False)
    while state.ss_class == 0:
        state.run_daily_dungeon()
        state.fuse_jewel()
    s_attempt_arr[i] = state.fuse_s_attempt
    ss_attempt_arr[i] = state.fuse_ss_attempt
    run_arr[i] = state.dungeon_run
    mesos_arr[i] = state.mesos_spent

print("average mesos per ss jewel: %.2f billion"%(np.average(mesos_arr)/1000))
print("average dungeon run per ss jewel: %.2f"%(np.average(run_arr)))
print("average s fuse attempt per ss jewel: %.2f"%(np.average(s_attempt_arr)))
print("average ss fuse attempt per ss jewel: %.2f"%(np.average(ss_attempt_arr)))
print("empirical probability of fusing SS: %.2f"%(1/np.average(ss_attempt_arr)))

average mesos per ss jewel: 1.80 billion
average dungeon run per ss jewel: 237.71
average s fuse attempt per ss jewel: 164.12
average ss fuse attempt per ss jewel: 19.73
empirical probability of fusing SS: 0.05


In [6]:
trials = 10000
s_attempt_arr = np.zeros(trials)
ss_attempt_arr = np.zeros(trials)
run_arr = np.zeros(trials)
mesos_arr = np.zeros(trials)

for i in range(trials):
    state = Jewel(use_orange=False, verbose=False)
    while state.ss_class == 0:
        state.run_daily_dungeon()
        state.fuse_jewel()
    s_attempt_arr[i] = state.fuse_s_attempt
    ss_attempt_arr[i] = state.fuse_ss_attempt
    run_arr[i] = state.dungeon_run
    mesos_arr[i] = state.mesos_spent
    
print("average mesos per ss jewel: %.2f billion"%(np.average(mesos_arr)/1000))
print("average dungeon run per ss jewel: %.2f"%(np.average(run_arr)))
print("average s fuse attempt per ss jewel: %.2f"%(np.average(s_attempt_arr)))
print("average ss fuse attempt per ss jewel: %.2f"%(np.average(ss_attempt_arr)))
print("empirical probability of fusing SS: %.2f"%(1/np.average(ss_attempt_arr)))

average mesos per ss jewel: 1.58 billion
average dungeon run per ss jewel: 388.57
average s fuse attempt per ss jewel: 268.51
average ss fuse attempt per ss jewel: 19.41
empirical probability of fusing SS: 0.05
